# Used Auto Price Recommendations and Fraud Flagging

The goal of this project was to train a model for a used car sales platform, which would be used both for recommending a sales price to new sellers as well as for flagging suspicious postings as potential fraud.

The dataset includes information about the cars (model, brand, number of kilometers on odometer, etc.) as well as about the postings (date posted, zip code, number of photos, etc.). The dataset contains several hundred thousand entries, so one of the criteria. As such, the criteria for evaluating model performance and choosing the final model were prediction accuracy (RMSE), training time, and prediction time.


*Note: Much of the code is commented because of the time it took to run, and because of the fact that I did not need to keep running the same code when working on/editing the project. Because of this, some results that would have been printed are instead in Markdown cells.*

### Импорты

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

from sklearn.dummy import DummyRegressor

import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb

import time

import optuna

import warnings
warnings.filterwarnings("ignore")


import phik

### Подготовка и просмотр данных

In [3]:
try:
    autos = pd.read_csv("datasets/autos.csv")
    
except:
    autos = pd.read_csv("/datasets/autos.csv")

In [4]:
display(autos.head())
autos.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Данные успешно загружены. Присуствуют пропуски в следующих столбцах:
    
- VehicleType  
- Gearbox
- Model
- FuelType
- NotRepaired
    
Также, надо будет поменять тип данных в столбце DateCreated. Поскольку точная дата не нужна, после переобразование такие значения на DateTime, стоит созать новые столбцы только с данными о месяце и году. 
    
Сразу видно, что некоторые столбцы не будут полезны для анализа, поскольку данные в них не может быть связаным с ценой машины. Из тех:
    
- DateCrawled
- LastSeen
    

*Преобразование столбцов с датой на datetime*

In [5]:
autos['DateCrawled'] = pd.to_datetime(autos['DateCrawled'], format='%Y-%m-%d %H:%M:%S')
autos['DateCreated'] = pd.to_datetime(autos['DateCreated'], format='%Y-%m-%d %H:%M:%S')
autos['LastSeen'] = pd.to_datetime(autos['LastSeen'], format='%Y-%m-%d %H:%M:%S')

*Просмотр и обработка столбцов с NaN*

In [6]:
print([col for col in autos.columns if len(autos[autos[col].notna() == True]) < len(autos)])

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']


В этом датасете, не хватает данных в пяти столбцах : 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired'

'Model' и 'NotRepaired' будет неправильно дополнять.

Для "Model", слишком много вариантов чтобы подбирать правильный. Модель машины может сильно повлиять на цену (целевой наш признак); такие строки составляют 5.56% датасета. С учётом всего этого, наверно лучше устранять такие строки.

Для 'NotRepaired' будет неправильно заменить пропуски потому, что нету способа предугадать то, была ли у машины проблема – даже самые новые машины иногда ломаются. Поэтому, было бы неправильно заменить с 'Yes' или 'No'. Однако, в этом столбце довольно много пропусков; поэтому, лучше заменить nan на 'unknown'.


In [7]:
print(autos['FuelType'].unique())
print(autos['Gearbox'].unique())
print(autos['VehicleType'].unique())

['petrol' 'gasoline' nan 'lpg' 'other' 'hybrid' 'cng' 'electric']
['manual' 'auto' nan]
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']


В оставшихся трёх столбцах с пропусками, наверно можно заменить пропуски, используя другие признаки.

Также, petrol и gasoline, это одно и то же, нужно их сложить вместе.

In [8]:
autos['FuelType'] = autos['FuelType'].replace('petrol', 'gasoline')

*Создание словарей для использования в замене пропусков*

In [9]:
brands = autos['Brand'].unique()
brandmodel = []
for brand in brands:
    brand_models_list = autos[autos['Brand']==brand]['Model'].unique()
    for model in brand_models_list:
        brandmodel.append((str(brand)+str(model)))

In [10]:
def brandmodel(row):
    return str(row['Brand'])+str(row['Model'])

In [11]:
autos['BrandModel'] = autos.apply(brandmodel, axis=1)

In [12]:
brandmodel_list = pd.Series(autos['BrandModel'].unique()).sort_values()

In [13]:
brandmodel_vehicle_type_dict = {}
for i_bm in brandmodel_list:
    brandmodel_vehicle_type_dict[i_bm] = (autos.query('(BrandModel == @i_bm) and (VehicleType == VehicleType)')["VehicleType"]).mode()[0]
    

In [14]:
brandmodel_fuel_dict = {}
for i_bm in brandmodel_list:
    brandmodel_fuel_dict[i_bm] = (autos.query('(BrandModel == @i_bm) and (FuelType == FuelType)')["FuelType"]).mode()[0]
    

*Замена пропусков в VehicleType и FuelType*

In [15]:
def fill_vehicletype(row):
    if ((row['VehicleType'] != row['VehicleType']) & 
        (row['Brand'] == row['Brand']) & 
        (row['Model'] == row['Model']) &
        (row['Model'] != 'other')):
            return brandmodel_vehicle_type_dict[row['BrandModel']]
    return row['VehicleType']

In [16]:
def fill_fueltype(row):
    if ((row['FuelType'] != row['FuelType']) & 
        (row['Brand'] == row['Brand']) & 
        (row['Model'] == row['Model']) &
        (row['Model'] != 'other')):
            return brandmodel_fuel_dict[row['BrandModel']]
    return row['VehicleType']

In [17]:
autos['VehicleType'] = autos.apply(fill_vehicletype, axis=1)

In [18]:
autos['FuelType'] = autos.apply(fill_fueltype, axis=1)

*Gearbox*

In [19]:
display(autos[
    (autos['Brand'] == 'volkswagen') &
    (autos['Model'] == 'golf')]['Gearbox'].value_counts(dropna=False))

print(f'''
Количество строк, в которых отсутствует информация про вид передачи: {len(autos[autos["Gearbox"].isna() == True])},
что является {len(autos[autos["Gearbox"].isna() == True])/len(autos)*100:.2f}% всего датасета.
''')

manual    24752
auto       3005
NaN        1475
Name: Gearbox, dtype: int64


Количество строк, в которых отсутствует информация про вид передачи: 19833,
что является 5.60% всего датасета.



Получается, что нет хорошего способа понять, какая передача у машины – оставляем.

*Устранение строк с значением NaN в критичном столбце Model*

In [20]:
autos = autos.dropna(subset=['Model'])

*Замена оставшихся пропусков в не критичных столбцах значением unknown*

In [21]:
autos['Gearbox'] = autos['Gearbox'].fillna(value='unknown')
autos['NotRepaired'] = autos['NotRepaired'].fillna(value='unknown')
autos['FuelType'] = autos['FuelType'].fillna(value='unknown')
autos['VehicleType'] = autos['VehicleType'].fillna(value='unknown')


In [22]:
display(autos.describe().T)

,count,mean,std,min,25%,50%,75%,max
Price,334664.0,4504.346790,4531.438572,0.0,1150.0,2800.0,6500.0,20000.0
RegistrationYear,334664.0,2003.923992,69.377219,1000.0,1999.0,2003.0,2008.0,9999.0
Power,334664.0,111.373195,185.156439,0.0,70.0,105.0,143.0,20000.0
Kilometer,334664.0,128562.588148,37205.926976,5000.0,125000.0,150000.0,150000.0,150000.0
RegistrationMonth,334664.0,5.806068,3.689145,0.0,3.0,6.0,9.0,12.0
NumberOfPictures,334664.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PostalCode,334664.0,50671.521616,25754.522388,1067.0,30419.0,49525.0,71229.0,99998.0


Если присмотреться к минимальным и максимальным значениям столбцов RegistrationYear, Price, и Power, явно есть какие-то ошибочные значения.

Удалим все строки где:

    - RegistrationYear < 1920   или   RegistrationYear > 2021
    - Price < 200
    - Power < 66  или  Power > 1479  (Я знаю крайне мало о лошадиной силе, но Гугл подсказал, что у Koenigsegg Regera – 1479hp и у Mitsubishi i-Miev – 66hp.)

Также, почему-то нету данных о количестве фото. Я перепроверил это, подумав, что я возможно случайно удалил все данные, но даже после того, как перескачал файл .csv, данных о фото не было. Поэтому, логично просто удалить этот столбец.

In [23]:
print(autos.query('Power < 150')['Power'].describe(), autos['Power'].max())

count    258023.000000
mean         83.418405
std          41.940611
min           0.000000
25%          60.000000
50%          90.000000
75%         116.000000
max         149.000000
Name: Power, dtype: float64 20000


In [24]:
autos = autos.query('(RegistrationYear > 1965 ) and (RegistrationYear < 2017) and (Price > 200) and (Power > 67) and (Power < 1436)')
autos = autos.drop(columns='NumberOfPictures')

In [25]:
display(autos.describe().T)

,count,mean,std,min,25%,50%,75%,max
Price,242898.0,5473.674501,4712.095783,202.0,1750.0,3900.0,7950.0,20000.0
RegistrationYear,242898.0,2003.431753,6.034967,1966.0,2000.0,2004.0,2008.0,2016.0
Power,242898.0,131.944261,54.407893,68.0,100.0,121.0,150.0,1433.0
Kilometer,242898.0,128675.637510,36571.864542,5000.0,125000.0,150000.0,150000.0,150000.0
RegistrationMonth,242898.0,6.082104,3.541460,0.0,3.0,6.0,9.0,12.0
PostalCode,242898.0,51804.859982,25868.571984,1067.0,31234.0,51103.0,72649.0,99998.0


*Создание двух столбцов с годом и месяцем создания анкеты относительно*

Стоимость машин зависит от времени, и из-за сезонности, и из-за экономических тенденций, поэтому надо создать столбцы, которые модели могут понимать и использовать.

In [26]:
autos['YearPosted'] = autos['DateCreated'].dt.year
autos['MonthPosted'] = autos['DateCreated'].dt.month
# autos = autos.drop(columns='DateCreated')

In [27]:
display(autos.info())
autos.describe(include = 'all')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 242898 entries, 2 to 354368
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        242898 non-null  datetime64[ns]
 1   Price              242898 non-null  int64         
 2   VehicleType        242898 non-null  object        
 3   RegistrationYear   242898 non-null  int64         
 4   Gearbox            242898 non-null  object        
 5   Power              242898 non-null  int64         
 6   Model              242898 non-null  object        
 7   Kilometer          242898 non-null  int64         
 8   RegistrationMonth  242898 non-null  int64         
 9   FuelType           242898 non-null  object        
 10  Brand              242898 non-null  object        
 11  NotRepaired        242898 non-null  object        
 12  DateCreated        242898 non-null  datetime64[ns]
 13  PostalCode         242898 non-null  int64   

None

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,PostalCode,LastSeen,BrandModel,YearPosted,MonthPosted
count,242898,242898.000000,242898,242898.000000,242898,242898.000000,242898,242898.000000,242898.000000,242898,242898,242898,242898,242898.000000,242898,242898,242898.000000,242898.000000
unique,202269,NaN,9,NaN,3,NaN,247,NaN,NaN,10,39,3,102,NaN,131230,294,NaN,NaN
top,2016-03-05 14:25:23,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,sedan,volkswagen,no,2016-04-03 00:00:00,NaN,2016-04-07 13:17:48,volkswagengolf,NaN,NaN
freq,5,NaN,81135,NaN,186229,NaN,22988,NaN,NaN,77704,50071,190594,9644,NaN,14,22988,NaN,NaN
first,2016-03-05 14:06:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-20 00:00:00,NaN,2016-03-05 14:15:08,NaN,NaN,NaN
last,2016-04-07 14:36:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-07 00:00:00,NaN,2016-04-07 14:58:51,NaN,NaN,NaN
mean,NaN,5473.674501,NaN,2003.431753,NaN,131.944261,NaN,128675.637510,6.082104,NaN,NaN,NaN,NaN,51804.859982,NaN,NaN,2015.999922,3.161500
std,NaN,4712.095783,NaN,6.034967,NaN,54.407893,NaN,36571.864542,3.541460,NaN,NaN,NaN,NaN,25868.571984,NaN,NaN,0.008844,0.378765
min,NaN,202.000000,NaN,1966.000000,NaN,68.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,1067.000000,NaN,NaN,2015.000000,1.000000
25%,NaN,1750.000000,NaN,2000.000000,NaN,100.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,31234.000000,NaN,NaN,2016.000000,3.000000


In [28]:
display(autos.columns)
autos_phik = autos.drop(columns=['DateCrawled', 'DateCreated', 'LastSeen', 'BrandModel'])
autos_phik.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'PostalCode', 'LastSeen', 'BrandModel',
       'YearPosted', 'MonthPosted'],
      dtype='object')

Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired',
       'PostalCode', 'YearPosted', 'MonthPosted'],
      dtype='object')

In [35]:
### Phi_K test
columns_dict = {
    'VehicleType': 'categorical',
    'RegistrationYear': 'interval',
    'Gearbox' : 'categorical',
    'Power' : 'interval',
    'Model' : 'categorical',
    'Kilometer' : 'interval',
    'RegistrationMonth' : 'categorical',
    'FuelType' : 'categorical',
    'Brand' : 'categorical',
    'NotRepaired' : 'categorical',
    'PostalCode' : 'categorical',
    'YearPosted' : 'interval',
    'MonthPosted' : 'categorical',
    'Price' : 'interval'
}

interval_cols = [col for col in columns_dict if columns_dict[col] == 'interval']
phik_corr = autos_phik.phik_matrix(interval_cols=interval_cols)

/Users/lilith/opt/anaconda3/lib/python3.9/site-packages/phik/bivariate.py:208: UserWarning: Many cells: 1991561. Are interval variables set correctly?
  warnings.warn('Many cells: {0:d}. Are interval variables set correctly?'.format(len(corr0)))


In [38]:
autos_phik.info()
display(phik_corr)
print(phik_corr.query('Price >= 0.29').index)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242898 entries, 2 to 354368
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              242898 non-null  int64 
 1   VehicleType        242898 non-null  object
 2   RegistrationYear   242898 non-null  int64 
 3   Gearbox            242898 non-null  object
 4   Power              242898 non-null  int64 
 5   Model              242898 non-null  object
 6   Kilometer          242898 non-null  int64 
 7   RegistrationMonth  242898 non-null  int64 
 8   FuelType           242898 non-null  object
 9   Brand              242898 non-null  object
 10  NotRepaired        242898 non-null  object
 11  PostalCode         242898 non-null  int64 
 12  YearPosted         242898 non-null  int64 
 13  MonthPosted        242898 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 27.8+ MB


,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode,YearPosted,MonthPosted
Price,1.000000,0.205391,0.674683,0.267941,0.324037,0.556214,0.347664,0.119948,0.292439,0.305863,0.329931,0.369643,NaN,0.021265
VehicleType,0.205391,1.000000,0.206696,0.287203,0.164724,0.909020,0.285441,0.086862,0.994139,0.593149,0.133772,0.345881,NaN,0.022981
RegistrationYear,0.674683,0.206696,1.000000,0.102919,0.113596,0.617841,0.449651,0.117455,0.298798,0.300642,0.208627,0.395740,NaN,0.013773
Gearbox,0.267941,0.287203,0.102919,1.000000,0.370770,0.605205,0.058269,0.146711,0.235506,0.502817,0.227570,0.294145,NaN,0.008732
Power,0.324037,0.164724,0.113596,0.370770,1.000000,0.533467,0.041812,0.028415,0.223840,0.290795,0.048553,0.159236,NaN,0.000000
Model,0.556214,0.909020,0.617841,0.605205,0.533467,1.000000,0.467564,0.166786,0.897043,0.998133,0.214814,0.907704,NaN,0.175118
Kilometer,0.347664,0.285441,0.449651,0.058269,0.041812,0.467564,1.000000,0.069244,0.209093,0.278011,0.226392,0.358345,NaN,0.017652
RegistrationMonth,0.119948,0.086862,0.117455,0.146711,0.028415,0.166786,0.069244,1.000000,0.178576,0.086655,0.308449,0.382111,NaN,0.019646
FuelType,0.292439,0.994139,0.298798,0.235506,0.223840,0.897043,0.209093,0.178576,1.000000,0.583240,0.204562,0.351501,NaN,0.023523
Brand,0.305863,0.593149,0.300642,0.502817,0.290795,0.998133,0.278011,0.086655,0.583240,1.000000,0.125452,0.654313,NaN,0.040137


Index(['Price', 'RegistrationYear', 'Power', 'Model', 'Kilometer', 'FuelType',
       'Brand', 'NotRepaired', 'PostalCode'],
      dtype='object')


Есть некоторые признаки, от которых лучше избавиться. Поскольку Model важный признак, буду опираться на нём чтобы выбрать признаки для устранения.

Признаки, с которыми у Model есть высокая коррелация:

- VehicleType
- Power
- FuelType
- Brand
- PostalCode

Из этих, у VehicleType, FuelType, и PostalCode самые низкая коррелация (~0.9). С учётом того, что PostalCode (почтовый индекс) и  VehicleType (тип автомобиля) могут сильно влиять на цену машины, я их оставлю. Я менее уверен насчёт значимости FuelType, но у него (их вышеперечисленных) самая низкая коррелация с Model, поэтому тоже его оставлю. Устраню тогда только Brand и Power.

Почему-то YearPosted выдал NaN. Я не совсем понимаю почему, поскольку у этого признака тип int64 и я включил его в список interval_cols, всё точно так же как с признаком RegistrationYear, с которым не было проблем. Я в любом случае бы удалил этот столбец, поэтому не стал сыскать причину. Также, я получил Warning о interval_cols, но тут тоже не совсем понимаю, почему. Правильно ли я назначил interval столбцы?


## Обучение моделей

Попробуем разные модели чтобы найти наиболее эффективую. 

Для сравнивания моделей, воспользуемся следующими критериями:

- точность модели – определена на основе метрике RMSE
- скорость работы
    - для обучения и для предсказывания
    - найдена с помощью time.time()

Датасет большой (350000+ строк), поэтому вопрос о скорости работы очень важен.

Модели, которые протестируем:

- sklearn.linear_model.LinearRegression
- sklearn.linear_model.Ridge
- sklearn.ensemble.RandomForestRegressor
- sklearn.ensemble.ExtraTreesRegressor
- lightgbm.LGBMRegressor
- catboost.CatBoostRegressor
- xgboost.XGBRegressor

In [39]:
cat_list = ['Model', 'NotRepaired', 'PostalCode', 'FuelType', 'Brand']

In [40]:
autos_df = autos.copy()

In [41]:
autos_features = autos.drop(columns=['Price', 'DateCrawled', 'DateCreated', 'LastSeen', 
                                     'BrandModel', 'YearPosted', 'Gearbox', 'VehicleType',
                                    'MonthPosted', 'RegistrationMonth' ])
autos_target = autos['Price']


*Кодирование признаков*

In [42]:
encoder = OrdinalEncoder()

autos_features[cat_list] = encoder.fit_transform(autos_features[cat_list])

display(autos_features.head())

,RegistrationYear,Power,Model,Kilometer,FuelType,Brand,NotRepaired,PostalCode
2,2004,163,115.0,125000,7.0,14.0,1.0,6922.0
3,2001,75,114.0,150000,6.0,37.0,0.0,6962.0
4,2008,69,99.0,90000,6.0,31.0,0.0,4161.0
5,1995,102,11.0,150000,5.0,2.0,2.0,2336.0
6,2004,109,8.0,150000,1.0,25.0,0.0,4585.0


*Разделение данных на треновочные и тестовые выборки*

In [43]:
train_features, test_valid_features, train_target, test_valid_target = (
    train_test_split(autos_features, autos_target, test_size=0.4, random_state=12345)
)

test_features, valid_features, test_target, valid_target = (
    train_test_split(test_valid_features, test_valid_target, test_size=0.5, random_state=12345)
)

*Масштабирование данных*

In [44]:
numeric = ['RegistrationYear', 'Power', 'Kilometer'] #есть другие численные признаки, но они категорические (н.п. месяц регистарции)

In [45]:
scaler = StandardScaler()
scaler.fit(train_features[numeric])
train_features[numeric] = scaler.transform(train_features[numeric])
test_features[numeric] = scaler.transform(test_features[numeric])
valid_features[numeric] = scaler.transform(valid_features[numeric])

*Сравнение разных моделей*

In [46]:
rmse_models = {}
train_time_models = {}
pred_time_models = {}
for model, name in zip([LinearRegression(), Ridge(random_state=12345), 
                        RandomForestRegressor(random_state=12345), ExtraTreesRegressor(random_state=12345),
                        CatBoostRegressor(verbose=False, random_state=12345), lgb.LGBMRegressor(random_state=12345), 
                        xgb.XGBRegressor(random_state=12345), DummyRegressor()], 
                       ['lin_reg', 'ridge', 'random_forest', 'extra_trees', 'cat_boost', 'lgbm_reg', 'xgb_reg', 'dummy']):
    begin = time.time()
    model.fit(train_features, train_target)
    end = time.time()
    train_time_models[name] = end-begin
    begin = time.time()
    pred = model.predict(test_features)
    end = time.time()
    rmse_models[name] = mse(test_target, pred, squared=False)
    pred_time_models[name] = end-begin
                  

In [47]:
models_test_df = pd.DataFrame.from_dict(rmse_models, orient="index")
models_test_df.columns = ['rmse']
models_test_df['train_time'] = pd.DataFrame.from_dict(train_time_models, orient="index").values
models_test_df['pred_time'] = pd.DataFrame.from_dict(pred_time_models, orient="index").values
models_test_df = models_test_df.sort_values(by='rmse')
display(models_test_df)


,rmse,train_time,pred_time
cat_boost,1673.363031,13.757743,0.036305
random_forest,1684.365312,45.234707,2.437342
xgb_reg,1688.791463,7.671959,0.060048
extra_trees,1740.062234,29.983985,4.725989
lgbm_reg,1765.601187,0.663993,0.151729
lin_reg,3162.576945,0.066852,0.006726
ridge,3162.576982,0.026069,0.001823
dummy,4727.099620,0.000527,0.000171


##### Подбор лучших гиперпараметров

В начале я пробовал найти лучшие гиперпараметры с помощью цикла. Такой подход правда способствовал улучшению модели, но он был очень неэффективен в этом. В итоге я использовал библиотеку Optuna чтобы найти лучшие гиперпараметры. Optuna занимается тем же, что я сам делал с циклами, но он многократно эффективнее и к тому же подбирает дробленные числа гораздо лучше, чем было бы возможно в цикле.

Я не стал искать лучшие гиперпараметры для каждой молели. С учётом времени обработки и результат RMSE, лучшие модели – CatBoostRegressor, XGBRegressor, и LGBMRegressor. У RandomForestRegressor самый лучший RMSE, но она также несколько раз медленнее чем перечисленные модели.

Когда я начал этот процесс, я планировал использовать Optuna дважды. В первый раз с меньшим количеством повторений (n_trials), чтобы понимать, какие гиперпараметры больше всего влияют на результаты; а второй раз чтобы получить результаты, уже используя найденные самые важные гиперпараметры. Однако, когда я повторно делал первый этап, я получал разные результаты (то есть, разные гиперпараметры). Поэтому, не стал использовать Optuna дважды в конце концов.

*LGBMRegressor*

In [48]:
# def objective_lgb(trial):
    
#     params = {
#         'boosting_type' : 'gbdt',
#         'n_estimators' : trial.suggest_int('n_estimators', 50, 1000, 50),
#         'learning_rate' : trial.suggest_float('learning_rate', 0.05, 0.2),
#         'max_depth' : trial.suggest_int('max_depth', 1, 20, 2),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 50, 1000, 50),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
#         "subsample": trial.suggest_float("subsample", 0.3, 1.0),
#         "subsample_freq": trial.suggest_int("subsample_freq", 1, 5),
#         'min_child_samples': trial.suggest_int('min_child_samples', 10, 300, 10),
#         'verbose' : -1
#     }
#     lgb_mod = lgb.LGBMRegressor(**params, random_state=12345)
#     lgb_mod.fit(train_features, train_target)
#     pred = lgb_mod.predict(test_features)
#     rmse = mse(test_target, pred, squared=False)
#     return rmse

# study_lgb = optuna.create_study(direction='minimize')
# study_lgb.optimize(objective_lgb, n_trials=50)


# print("Number of finished trials: {}".format(len(study_lgb.trials)))

# print("Best trial:")
# trial_lgb = study_lgb.best_trial

# print("  Value: {}".format(trial_lgb.value))

# print("  Params: ")
# for key, value in trial_lgb.params.items():
#     print("    {}: {}".format(key, value))


# display(optuna.importance.get_param_importances(study_lgb))

[I 2022-06-08 08:36:59,926] A new study created in memory with name: no-name-b666cde5-718c-42cd-b574-2749c9a43859
[I 2022-06-08 08:37:04,327] Trial 0 finished with value: 1659.5757318020653 and parameters: {'n_estimators': 500, 'learning_rate': 0.0955813798812167, 'max_depth': 7, 'reg_alpha': 2.5161949244912894e-08, 'reg_lambda': 6.93810614631105, 'num_leaves': 750, 'colsample_bytree': 0.9165368476762207, 'subsample': 0.8170677516475304, 'subsample_freq': 1, 'min_child_samples': 80}. Best is trial 0 with value: 1659.5757318020653.
[I 2022-06-08 08:37:24,277] Trial 1 finished with value: 1622.2118156672682 and parameters: {'n_estimators': 700, 'learning_rate': 0.07357116822232429, 'max_depth': 11, 'reg_alpha': 0.10963247588044783, 'reg_lambda': 0.00025373214950172757, 'num_leaves': 450, 'colsample_bytree': 0.9792939488890438, 'subsample': 0.47660212519791345, 'subsample_freq': 1, 'min_child_samples': 10}. Best is trial 1 with value: 1622.2118156672682.
[I 2022-06-08 08:37:33,421] Trial 

Number of finished trials: 50
Best trial:
  Value: 1595.5027527736993
  Params: 
    n_estimators: 900
    learning_rate: 0.08176805647342944
    max_depth: 13
    reg_alpha: 0.04315868514041527
    reg_lambda: 0.20771623873858824
    num_leaves: 950
    colsample_bytree: 0.6354518911703311
    subsample: 0.94745597016541
    subsample_freq: 3
    min_child_samples: 30


OrderedDict([('max_depth', 0.6476995818461116),
             ('n_estimators', 0.16794550722733706),
             ('learning_rate', 0.07718389349715331),
             ('colsample_bytree', 0.041251670156001334),
             ('subsample', 0.020291704965599285),
             ('min_child_samples', 0.01875194416207544),
             ('reg_alpha', 0.016396536517421158),
             ('num_leaves', 0.00585502046430303),
             ('subsample_freq', 0.004572818141727508),
             ('reg_lambda', 5.13230222701586e-05)])

Best trial:
    
  Value: 1595.5027527736993
    
  Params: 
    
- n_estimators: 900
- learning_rate: 0.08176805647342944
- max_depth: 13
- reg_alpha: 0.04315868514041527
- reg_lambda: 0.20771623873858824
- num_leaves: 950
- colsample_bytree: 0.6354518911703311
- subsample: 0.94745597016541
- subsample_freq: 3
- min_child_samples: 30
    


*CatBoostRegressor*

In [50]:
train_features[cat_list] = train_features[cat_list].astype('int32')
test_features[cat_list] = test_features[cat_list].astype('int32')
valid_features[cat_list] = valid_features[cat_list].astype('int32')

In [51]:
# def objective_cat(trial):
    
#     params = {
#         'iterations' : trial.suggest_int('iterations', 5, 105, 10),
#         'depth' : trial.suggest_int('depth', 1, 16),
#         'learning_rate' : trial.suggest_float('learning_rate', 0.1, 1),
#         'iterations' : trial.suggest_int('iterations', 5, 1005, 100),
#         'subsample' : trial.suggest_float('subsample', 0.6, 1.0),
# #         'n_estimators' : trial.suggest_int('n_estimators', 60, 100, 20),
# #         'num_leaves': trial.suggest_int('num_leaves', 20, 50, 5),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'cat_features' : cat_list,
#         'early_stopping_rounds' : 70,
#         'loss_function' : 'RMSE',
#         'silent' : True
#     }
    
#     cat_mod = CatBoostRegressor(**params, random_state=12345)
#     cat_mod.fit(train_features, train_target)
#     pred = cat_mod.predict(test_features)
#     rmse = mse(test_target, pred, squared=False)
#     return rmse

# study_cat = optuna.create_study(direction='minimize')
# study_cat.optimize(objective_cat, n_trials=50)


# print("Number of finished trials: {}".format(len(study_cat.trials)))

# print("Best trial:")
# trial_cat = study_cat.best_trial

# print("  Value: {}".format(trial_cat.value))

# print("  Params: ")
# for key, value in trial_cat.params.items():
#     print("    {}: {}".format(key, value))


# display(optuna.importance.get_param_importances(study_cat))




[I 2022-06-08 09:02:06,732] A new study created in memory with name: no-name-f27d5d5f-d53c-49bb-8745-e0b9918c0e67
[I 2022-06-08 09:02:14,748] Trial 0 finished with value: 1717.204811575568 and parameters: {'iterations': 45, 'depth': 13, 'learning_rate': 0.6718425637486675, 'subsample': 0.6003067755430331, 'min_child_samples': 10}. Best is trial 0 with value: 1717.204811575568.
[I 2022-06-08 09:02:36,857] Trial 1 finished with value: 1740.2039082134188 and parameters: {'iterations': 75, 'depth': 14, 'learning_rate': 0.7025458373362815, 'subsample': 0.9245632488103972, 'min_child_samples': 81}. Best is trial 0 with value: 1717.204811575568.
[I 2022-06-08 09:02:38,139] Trial 2 finished with value: 1862.4586388157893 and parameters: {'iterations': 25, 'depth': 8, 'learning_rate': 0.25716124957533243, 'subsample': 0.9418131175384887, 'min_child_samples': 11}. Best is trial 0 with value: 1717.204811575568.
[I 2022-06-08 09:02:45,299] Trial 3 finished with value: 1696.2908954362888 and parame

Number of finished trials: 50
Best trial:
  Value: 1664.4740556840884
  Params: 
    iterations: 105
    depth: 12
    learning_rate: 0.34502285072578975
    subsample: 0.7675931036604555
    min_child_samples: 12


OrderedDict([('depth', 0.5218057814171859),
             ('iterations', 0.34486722338861886),
             ('subsample', 0.07636264716324503),
             ('learning_rate', 0.044385252771976864),
             ('min_child_samples', 0.012579095258973468)])

Best trial:
    
Value: 1664.4740556840884

Params: 

- iterations: 105
- depth: 12
- learning_rate: 0.34502285072578975
- subsample: 0.7675931036604555
- min_child_samples: 12


*XGBRegressor*

In [52]:
# def objective_xgb(trial):
    
#     params = {
#         'n_estimators' : trial.suggest_int('n_estimators', 5, 150),
#         'max_depth' : trial.suggest_int('max_depth', 2, 16),
#         'max_leaves' : trial.suggest_int('max_leaves', 30, 100),
#         'learning_rate' : trial.suggest_float('learning_rate', 0.1, 1),
#         'reg_alpha' : trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
#         'reg_lambda' : trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
# #         'objective' : 'RMSE',
#         'verbosity' : 0,
#         'booster' : 'gbtree'
#     }
    
#     xgb_mod = xgb.XGBRegressor(**params, random_state=12345)
#     xgb_mod.fit(train_features, train_target)
#     pred = xgb_mod.predict(test_features)
#     rmse = mse(test_target, pred, squared=False)
#     return rmse

# study_xgb = optuna.create_study(direction='minimize')
# study_xgb.optimize(objective_xgb, n_trials=50)


# print("Number of finished trials: {}".format(len(study_xgb.trials)))

# print("Best trial:")
# trial_xgb = study_xgb.best_trial

# print("  Value: {}".format(trial_xgb.value))

# print("  Params: ")
# for key, value in trial_xgb.params.items():
#     print("    {}: {}".format(key, value))


# display(optuna.importance.get_param_importances(study_xgb))




[I 2022-06-08 09:22:46,155] A new study created in memory with name: no-name-09730f1e-e8f9-4ca0-a0cf-0a8db92d97bd
[I 2022-06-08 09:22:51,788] Trial 0 finished with value: 1764.5734976425888 and parameters: {'n_estimators': 125, 'max_depth': 4, 'max_leaves': 91, 'learning_rate': 0.9467143674570518, 'reg_alpha': 7.840986043963627e-08, 'reg_lambda': 6.812301745153465e-08}. Best is trial 0 with value: 1764.5734976425888.
[I 2022-06-08 09:22:56,679] Trial 1 finished with value: 2006.5340520130364 and parameters: {'n_estimators': 18, 'max_depth': 13, 'max_leaves': 94, 'learning_rate': 0.10365650713274917, 'reg_alpha': 1.004276324601657e-05, 'reg_lambda': 5.563694876689111e-05}. Best is trial 0 with value: 1764.5734976425888.
[I 2022-06-08 09:22:57,373] Trial 2 finished with value: 2283.1084560383333 and parameters: {'n_estimators': 27, 'max_depth': 2, 'max_leaves': 58, 'learning_rate': 0.23891773756578155, 'reg_alpha': 7.142298922302561, 'reg_lambda': 0.35399051303991713}. Best is trial 0 wi

Number of finished trials: 50
Best trial:
  Value: 1603.8272533091256
  Params: 
    n_estimators: 141
    max_depth: 12
    max_leaves: 50
    learning_rate: 0.11201604434674199
    reg_alpha: 0.02705831024214185
    reg_lambda: 5.055856560216664


OrderedDict([('n_estimators', 0.7807181082220772),
             ('learning_rate', 0.07791204746801371),
             ('max_depth', 0.07228356761326553),
             ('reg_alpha', 0.05867775879909895),
             ('max_leaves', 0.006098454173730954),
             ('reg_lambda', 0.004310063723813723)])

Best trial:
    
  Value: 1603.8272533091256
    
  Params: 
    
- n_estimators: 141
- max_depth: 12
- max_leaves: 50
- learning_rate: 0.11201604434674199
- reg_alpha: 0.02705831024214185
- reg_lambda: 5.055856560216664



## Анализ моделей

*Сравнение лучших моделей*

In [53]:
# best_rmses = {}
# best_train_times = {}
# best_pred_times = {}


# ### CatBoost

# best_cat = CatBoostRegressor(**study_cat.best_params, cat_features=cat_list, 
#                              early_stopping_rounds=70, loss_function='RMSE',
#                              silent=True, random_state=12345)

# begin = time.time()
# best_cat.fit(train_features, train_target)
# end = time.time()

# best_train_times['CatBoost'] = end-begin

# begin = time.time()
# pred = best_cat.predict(valid_features)
# end = time.time()

# rmse = mse(valid_target, pred, squared=False)
# best_rmses['CatBoost'] = rmse
# best_pred_times['CatBoost'] = end-begin


# ### LGBMRegressor

# best_lgb = lgb.LGBMRegressor(**study_lgb.best_params, boosting_type='gbdt', verbose=-1, random_state=12345)

# begin = time.time()
# best_lgb.fit(train_features, train_target)
# end = time.time()

# best_train_times['LGBR'] = end-begin

# begin = time.time()
# pred = best_lgb.predict(valid_features)
# end = time.time()

# rmse = mse(valid_target, pred, squared=False)
# best_rmses['LGBR'] = rmse
# best_pred_times['LGBR'] = end-begin

# ### XGBoost

# best_xgb = xgb.XGBRegressor(**study_xgb.best_params, verbosity=0, booster='gbtree', random_state=12345)

# begin = time.time()
# best_xgb.fit(train_features, train_target)
# end = time.time()

# best_train_times['XGBoost'] = end-begin

# begin = time.time()
# pred = best_xgb.predict(valid_features)
# end = time.time()

# rmse = mse(valid_target, pred, squared=False)
# best_rmses['XGBoost'] = rmse
# best_pred_times['XGBoost'] = end-begin


In [54]:
# best_models_df = pd.DataFrame.from_dict(best_rmses, orient="index")
# best_models_df = best_models_df.merge(pd.DataFrame.from_dict(best_train_times, orient="index"), how='left', on=best_models_df.index)
# best_models_df.columns = ['model', 'rmse', 'train_time']
# best_models_df['pred_time'] = pd.DataFrame.from_dict(best_pred_times, orient="index").values
# best_models_df = best_models_df.sort_values(by='rmse')
# display(best_models_df)

,model,rmse,train_time,pred_time
1,LGBR,1585.631124,26.797263,12.521861
2,XGBoost,1591.541284,42.694728,0.644562
0,CatBoost,1646.009119,11.697080,0.089709



    model	rmse	train_time	pred_time

    LGBR	1585.631124	26.797263	12.521861

    XGBoost	1591.541284	42.694728	0.644562

    CatBoost	1646.009119	11.697080	0.089709
    


### Выводы

Цель этого проекта была тем, чтобы найти лучшую модель для предсказания стоимость машин. Главные критерии были:

- точность предсказаний
- время обучения
- скорость предсказывания
   


Лучшая модель: LGBRegressor
    
Время обучение: 26.797263 

Время предсказывания: 12.521861

RMSE: 1585.631124 
    

  Params: 
    
- n_estimators: 900
- learning_rate: 0.08176805647342944
- max_depth: 13
- reg_alpha: 0.04315868514041527
- reg_lambda: 0.20771623873858824
- num_leaves: 950
- colsample_bytree: 0.6354518911703311
- subsample: 0.94745597016541
- subsample_freq: 3
- min_child_samples: 30
